In [ ]:
# installations
%pip install langchain | tail -n 1
%pip install langchain-ibm | tail -n 1
%pip install langchain-community | tail -n 1
%pip install ibm-watsonx-ai | tail -n 1
%pip install chromadb | tail -n 1
%pip install tiktoken | tail -n 1
%pip install bs4 | tail -n 1

In [ ]:
# imports
import getpass

from langchain_ibm import WatsonxEmbeddings, WatsonxLLM
from langchain.vectorstores import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.prompts import PromptTemplate
from langchain.tools import tool
from langchain.tools.render import render_text_description_and_args
from langchain.agents.output_parsers import JSONAgentOutputParser
from langchain.agents.format_scratchpad import format_log_to_str
from langchain.agents import AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain_core.runnables import RunnablePassthrough
from ibm_watsonx_ai.foundation_models.utils.enums import EmbeddingTypes
from ibm_watsonx_ai.metanames import GenTextParamsMetaNames as GenParams

In [ ]:
#Setting Up API key and Project ID
credentials = {
    "url": "https://us-south.ml.cloud.ibm.com",
    "apikey": getpass.getpass("Please enter your watsonx.ai Runtime API key (hit enter): ")
}

project_id = getpass.getpass("Please enter your project ID (hit enter): ")

In [ ]:
#Initialization of basic tools with no agents
llm = WatsonxLLM(
    model_id= "ibm/granite-3-8b-instruct", 
    url=credentials.get("url"),
    apikey=credentials.get("apikey"),
    project_id=project_id,
    params={
        GenParams.DECODING_METHOD: "greedy",
        GenParams.TEMPERATURE: 0,
        GenParams.MIN_NEW_TOKENS: 5,
        GenParams.MAX_NEW_TOKENS: 250,
        GenParams.STOP_SEQUENCES: ["Human:", "Observation"],
    },
)

In [ ]:
#prompt template in case you want to ask multiple questions.
template = "Answer the {query} accurately. If you do not know the answer, simply say you do not know."
prompt = PromptTemplate.from_template(template)
#set up a chain with our prompt and our LLM. This allows the generative model to produce a response.
agent = prompt | llm

In [ ]:
#Ask the Agent a question
agent.invoke({"query": "What is the optimal High Jump technique"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "What was the takeoff angle of my knee?"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "What was the takeoff angle of my knee during my jump?"})

In [ ]:
#Asking another set of Questions
agent.invoke({"query": "If you had access to my jump data via MediaPipe, would you be able to analyze the physical movements?"})

In [ ]:
#Websites that data is extracted from... Data Source/knowledge base
urls = [
    "https://en.wikipedia.org/wiki/High_jump",
    "https://worldathletics.org/disciplines/jumps/high-jump",
    "https://athleticssa.org.za/SportsInfo/Coaching-High-Jump.pdf",
    "https://coachathletics.com.au/coaching-education/how-to-coach-the-fosbury-flop-drill-progression-for-teaching-high-jump"

]

In [ ]:
#Loading documents using LangChain WebBaseLoader from urls listed
docs = [WebBaseLoader(url).load() for url in urls]
docs_list = [item for sublist in docs for item in sublist]
docs_list[0]

In [ ]:
#Text Splitter
text_splitter = RecursiveCharacterTextSplitter.from_tiktoken_encoder(
    chunk_size=250, chunk_overlap=0
)
doc_splits = text_splitter.split_documents(docs_list)

In [ ]:
#Embedding models
embeddings = WatsonxEmbeddings(
    model_id=EmbeddingTypes.IBM_SLATE_30M_ENG.value,
    url=credentials["url"],
    apikey=credentials["apikey"],
    project_id=project_id,
)

In [ ]:
#Storage of embedded documents using Chroma DB
vectorstore = Chroma.from_documents(
    documents=doc_splits,
    collection_name="agentic-rag-chroma",
    embedding=embeddings,
)

In [ ]:
#Access Information in vector store
retriever = vectorstore.as_retriever()

In [ ]:
@tool
def get_HighJump_Info(question: str, metrics: Optional[dict] = None) -> str:
    """
    Get context about High Jump including technique, approach, optimal velocity, hip height, and body angles.
    Optionally include MediaPipe pose estimation metrics to evaluate form based on key points.

    Expected metrics keys (optional): hip_angle, knee_angle, takeoff_velocity, max_hip_height, keypoints
    """
    context = retriever.invoke(question)
    feedback = []

    # Only analyze KPIs if metrics are provided
    if metrics:
        hip_angle = metrics.get("hip_angle", None)
        knee_angle = metrics.get("knee_angle", None)
        velocity = metrics.get("takeoff_velocity", None)
        hip_height = metrics.get("max_hip_height", None)

        # --- KPI Evaluations ---
        if hip_angle is not None:
            if hip_angle < 130:
                feedback.append(f"Hip angle of {hip_angle}° is too low; work on knee drive.")
            else:
                feedback.append(f"Hip angle of {hip_angle}° is strong.")

        if knee_angle is not None:
            if knee_angle < 160:
                feedback.append(f"Knee angle of {knee_angle}° suggests limited extension.")
            else:
                feedback.append(f"Knee angle of {knee_angle}° shows good extension.")

        if velocity is not None:
            if velocity < 3.5:
                feedback.append(f"Takeoff velocity of {velocity} m/s is below optimal. Increase run-up speed.")
            else:
                feedback.append(f"Takeoff velocity of {velocity} m/s is sufficient.")

        if hip_height is not None:
            if hip_height < 0.85:
                feedback.append(f"Max hip height of {hip_height}m is low. Emphasize vertical lift at takeoff.")
            else:
                feedback.append(f"Hip height of {hip_height}m is excellent.")

        # Optionally include raw pose keypoints
        keypoints = metrics.get("keypoints", None)
        if keypoints:
            feedback.append(f"Pose Keypoints received for analysis: {list(keypoints.keys())[:5]}... (truncated)")

    # Final response
    return "\n".join([
        " Technique Context:\n" + context,
        "\n KPI Feedback:\n" + ("\n".join(feedback) if feedback else "No metrics provided.")
    ])
tools = [get_HighJump_Info]


In [ ]:
system_prompt = """You are a Virtual High Jump Coach. Your role is to assist athletes by providing precise and constructive feedback about their high jump performance. You have access to the following tools: {tools}

Use a JSON blob to select a tool by providing an "action" key (the tool name) and an "action_input" key (the input for that tool).  
Valid "action" values include: "Final Answer" or {tool_names}

Each tool action must be wrapped in a JSON object like this:

{{
  "action": $TOOL_NAME,
  "action_input": $INPUT
}}

Always follow this reasoning pattern:
Question: Athlete's question or comment  
Thought: Reflect on the athlete’s movement data, technique reference, or biomechanical insight  
Action:

$JSON_BLOB

Observation: Response or result from the tool  
... (repeat Thought/Action/Observation N times if needed)  
Thought: I now understand how to best guide the athlete  
Action:

{{
  "action": "Final Answer",
  "action_input": "Final coaching advice or explanation"
}}
"""

In [ ]:
human_prompt = """{input}

{agent_scratchpad}

(reminder to always respond in a JSON blob)
"""

In [ ]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history", optional=True),
        ("human", human_prompt),
    ]
)

In [ ]:
#finalize our prompt template by adding the tool names, descriptions and arguments using a partial prompt template.
#This allows the agent to access the information pertaining to each tool
#including the intended use cases and also means we can add and remove tools without altering our entire prompt template.
prompt = prompt.partial(
    tools=render_text_description_and_args(list(tools)),
    tool_names=", ".join([t.name for t in tools]),
)

In [ ]:
#Setting up agents memory
#use LangChain's ConversationBufferMemory() as a means of memory storage.
memory = ConversationBufferMemory()

In [ ]:
#set up a chain with our agent's scratchpad, memory, prompt and the LLM.
#The AgentExecutor class is used to execute the agent. It takes the agent, its tools, error handling approach, verbose parameter and memory as parameters.
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_log_to_str(x["intermediate_steps"]),
        chat_history=lambda x: memory.chat_memory.messages,
    )
    | prompt
    | llm
    | JSONAgentOutputParser()
)

agent_executor = AgentExecutor(
    agent=chain, tools=tools, handle_parsing_errors=True, verbose=True, memory=memory
)

In [ ]:
#We are now able to ask the agent questions.
agent_executor.invoke({"input":"Question})"